In [2]:

import requests
import sqlite3
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

class FootballDatabase:
    def __init__(self, database_path, base_url="https://v3.football.api-sports.io/"):
        self.conn = sqlite3.connect(database_path)
        self.cursor = self.conn.cursor()
        self.api_key = os.getenv("x-rapidapi-key")
        self.base_url = base_url
        self.headers = {"x-rapidapi-key": self.api_key}

    def initialize_database(self):
        # Create tables if they don't exist
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS teams (
            team_id INTEGER PRIMARY KEY,
            name TEXT,
            league_id INTEGER,
            season INTEGER
        )
        ''')

        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS fixtures (
            fixture_id INTEGER PRIMARY KEY,
            date TEXT,  
            date_only TEXT,  
            time_only TEXT,  
            timezone TEXT,
            venue_name TEXT,
            venue_city TEXT,
            referee TEXT,
            status TEXT,
            league_id INTEGER,
            league_season INTEGER,
            league_round TEXT,
            home_team TEXT,
            away_team TEXT,
            home_goals INTEGER,
            away_goals INTEGER,
            home_team_id INTEGER,
            away_team_id INTEGER
        )
        ''')
        self.conn.commit()

    def fetch_data(self, endpoint, params):
        # Fetches data from the API
        response = requests.get(self.base_url + endpoint, headers=self.headers, params=params)
        response.raise_for_status()
        return response.json()

    def load_league_data(self, league_id, league_name):
        # Fetch teams and fixtures for a specific league
        print(f"Processing {league_name}...")

        # Fetch seasons for the league
        league_info = self.fetch_data("leagues", {"id": league_id}).get('response', [])
        if not league_info:
            print(f"No league information found for {league_name}. Skipping.")
            return

        seasons = league_info[0]['seasons']
        for season_info in seasons:
            season = season_info['year']
            print(f" - Season: {season}")

            # Fetch teams for the current season
            teams = self.fetch_data("teams", {"league": league_id, "season": season}).get('response', [])
            team_data = [(team['team']['id'], team['team']['name'], league_id, season) for team in teams]

            # Insert teams into the database
            self.cursor.executemany('''
                INSERT OR REPLACE INTO teams (team_id, name, league_id, season)
                VALUES (?, ?, ?, ?)
            ''', team_data)

            # Fetch fixtures for the current season
            fixtures = self.fetch_data("fixtures", {"league": league_id, "season": season}).get('response', [])
            fixture_data = []
            for fixture in fixtures:
                raw_datetime = fixture['fixture']['date']
                stripped_datetime = raw_datetime.split('+')[0]
                date_only, time_only = stripped_datetime.split('T')

                fixture_data.append((
                    fixture['fixture']['id'],
                    raw_datetime,
                    date_only,
                    time_only,
                    fixture['fixture']['timezone'],
                    fixture['fixture']['venue']['name'] if fixture['fixture']['venue'] else None,
                    fixture['fixture']['venue']['city'] if fixture['fixture']['venue'] else None,
                    fixture['fixture']['referee'],
                    fixture['fixture']['status']['long'],
                    league_id,
                    season,
                    fixture['league']['round'],
                    fixture['teams']['home']['name'],
                    fixture['teams']['away']['name'],
                    fixture['goals']['home'],
                    fixture['goals']['away'],
                    fixture['teams']['home']['id'],
                    fixture['teams']['away']['id']
                ))

            # Insert fixtures into the database
            self.cursor.executemany('''
                INSERT OR REPLACE INTO fixtures (
                    fixture_id, date, date_only, time_only, timezone, venue_name, venue_city, referee, 
                    status, league_id, league_season, league_round, home_team, away_team, 
                    home_goals, away_goals, home_team_id, away_team_id
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', fixture_data)

        print(f"{league_name} data added to the database.")

    def close_connection(self):
        self.conn.commit()
        self.conn.close()

# Main execution
if __name__ == "__main__":
    DATABASE_PATH = "./data/football_data.db"

    LEAGUE_IDS = {
        'Premier League': 39,
        'Championship': 40,
        'League One': 41,
        'League Two': 42
    }

    db = FootballDatabase(DATABASE_PATH)
    db.initialize_database()

    for league_name, league_id in LEAGUE_IDS.items():
        db.load_league_data(league_id, league_name)

    db.close_connection()
    print("Database setup complete.")


Processing Premier League...
 - Season: 2010
 - Season: 2011
 - Season: 2012
 - Season: 2013
 - Season: 2014
 - Season: 2015
 - Season: 2016
 - Season: 2017
 - Season: 2018
 - Season: 2019
 - Season: 2020
 - Season: 2021
 - Season: 2022
 - Season: 2023
 - Season: 2024
Premier League data added to the database.
Processing Championship...
 - Season: 2011
 - Season: 2012
 - Season: 2013
 - Season: 2014
 - Season: 2015
 - Season: 2016
 - Season: 2017
 - Season: 2018
 - Season: 2019
 - Season: 2020
 - Season: 2021
 - Season: 2022
 - Season: 2023
 - Season: 2024
Championship data added to the database.
Processing League One...
 - Season: 2011
 - Season: 2012
 - Season: 2013
 - Season: 2014
 - Season: 2015
 - Season: 2016
 - Season: 2017
 - Season: 2018
 - Season: 2019
 - Season: 2020
 - Season: 2021
 - Season: 2022
 - Season: 2023
 - Season: 2024
League One data added to the database.
Processing League Two...
 - Season: 2011
 - Season: 2012
 - Season: 2013
 - Season: 2014
 - Season: 2015
 - 